In [6]:
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_algorithms import QAOA
from qiskit.primitives import Sampler
from qiskit_algorithms.optimizers import COBYLA
from qiskit_optimization import QuadraticProgram

In [ ]:
def solve_multi_state_knapsack(profit_values, cost_values, max_total_cost):
    """
    Solve a multi-state knapsack problem using a classical backtracking algorithm.
    
    Args:
        profit_values (2d list of float): Profit values for the possible states.
        cost_values (2d list of float): Cost values for the possible states.
        max_total_cost (float): Maximum allowable total cost.
        
    Returns:
        list: The optimal selection of states for each item.
        float: The maximum total profit.

        
    # len(profit_values) == len(cost_values)
    # len(profit_values[0]) == len(cost_values[0])

    """
    n = len(profit_values[0])  # Number of items
    num_states = len(profit_values)
    
    def backtrack(index, current_cost, current_profit, current_solution):
        """
        Recursive backtracking function to search for the optimal solution.
        
        Args:
            index (int): Current item index being processed.
            current_cost (float): Current total cost of the solution path.
            current_profit (float): Current total profit of the solution path.
            current_solution (list of int): Current list of selected states for items.
        
        Returns:
            list: The best solution (state selection) for the current path.
            float: The total profit of the best solution.
        """
        if index == n:  # Base case: all items are processed
            return current_solution, current_profit
        
        best_solution, best_profit = None, -float('inf')
        
        for state in range(num_states):  # possible states for each item
            new_cost = current_cost + cost_values[state][index]
            new_profit = current_profit + profit_values[state][index]
            
            if new_cost <= max_total_cost:  # Prune paths exceeding the cost limit
                solution, profit = backtrack(
                    index + 1, 
                    new_cost, 
                    new_profit, 
                    current_solution + [state]
                )
                
                if solution and profit > best_profit:
                    best_solution, best_profit = solution, profit
        
        return best_solution, best_profit

    # Start the backtracking process from index 0
    solution, max_profit = backtrack(0, 0, 0, [])
    return solution, max_profit

In [8]:

def formulate_qp_multi_state_knapsack(profit_values, cost_values, max_total_cost):
    """
    Formulate the multi-state knapsack problem as a quadratic program using Qiskit's QuadraticProgram.
    
    Args:
        profit_values (2d list of float): Profit values for the possible states.
        cost_values (2d list of float): Cost values for the possible states.
        max_total_cost (float): Maximum allowable total cost.
    
    Returns:
        QuadraticProgram: The formulated quadratic program for QAOA-based solution.

        
    # len(profit_values) == len(cost_values)
    # len(profit_values[0]) == len(cost_values[0])

    """
    
    n = len(profit_values[0])  # Number of items
    num_states = len(profit_values)  # Number of possible states per item

    qp = QuadraticProgram("Multi-State Knapsack Problem")

    # Define binary variables for each item's possible state (states per item)
    for i in range(n):
        for s in range(num_states):
            qp.binary_var(name=f"state_{s}_{i}") 

    # Objective function: maximize profits
    linear_terms = {}
    for i in range(n):
        for s in range(num_states):
            linear_terms[f"state_{s}_{i}"] = profit_values[s][i] 

    qp.maximize(linear=linear_terms)

    # Constraint 1: Each item must be assigned to exactly one state
    for i in range(n):
        qp.linear_constraint(
            linear={f"state_{s}_{i}": 1 for s in range(num_states)},
            sense="==",
            rhs=1,
            name=f"state_assignment_constraint_{i}",
        )

    # Constraint 2: Total cost must not exceed the maximum allowable cost
    cost_terms = {}
    for i in range(n):
        for s in range(num_states):
            cost_terms[f"state_{s}_{i}"] = cost_values[s][i]

    qp.linear_constraint(
        linear=cost_terms,
        sense="<=",
        rhs=max_total_cost,
        name="total_cost_constraint",
    )

    return qp



In [ ]:
def solve_qp_with_qiskit(profit_values, cost_values, max_total_cost):
    """
    Solve the quadratic program using Qiskit's optimization solvers.

    Returns:
        dict: Solution details including selected items and total revenue.
    """
    qp = formulate_qp_multi_state_knapsack(profit_values, cost_values, max_total_cost)

    print(qp.prettyprint())
    
    qaoa = QAOA(sampler=Sampler(), optimizer=COBYLA())
    
    optimizer = MinimumEigenOptimizer(qaoa)

    result = optimizer.solve(qp)
    solution = result.x
    
    # Extract results
    total_profit = 0
    assignments = []

    
    n = len(profit_values[0])  # Number of items
    num_states = len(profit_values)  # Number of possible states per item


    for i in range(n):
        for s in range(num_states):
            if solution[num_states * i + s] == 1:
                total_profit += profit_values[s][i]
                assignments.append(s)


    return {
        "assignments": assignments,
        "total_profit": total_profit,
        "ansatz": qaoa.ansatz
    }

In [10]:
# L1 = [3, 5, 2, 4]
# L2 = [7, 1, 6, 8]
# L3 = [8, 2, 4, 5]
# C1 = [2, 3, 5, 4]
# C2 = [4, 2, 1, 5]
# C3 = [3, 4, 2, 6]


profits = [[3, 5, 2, 4],
           [7, 1, 6, 8],
           [8, 2, 4, 5]]
costs = [[2, 3, 5, 4],
         [4, 2, 1, 5],
         [3, 4, 2, 6]]


max_total_cost = 10


# Classical solution
solution, max_profit = solve_multi_state_knapsack(profits, costs, max_total_cost)
print(f"Optimal state selection: {solution}, Maximum profit: {max_profit}")

# Quantum solution
result = solve_qp_with_qiskit(profits, costs, max_total_cost)
print(f"Optimal state selection: {result["assignments"]}, Maximum profit: {result["total_profit"]}")

Optimal state selection: [2, 1, 1, 0], Maximum profit: 19


C:\Users\forca\AppData\Local\Temp\ipykernel_7636\192316337.py:10: DeprecationWarning: The class ``qiskit.primitives.sampler.Sampler`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseSamplerV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Sampler` class is `StatevectorSampler`.
  qaoa = QAOA(sampler=Sampler(), optimizer=COBYLA())


Optimal state selection: [2, 1, 1, 0], Maximum profit: 19
